In [28]:
from gymnasium import spaces
from ray.rllib.env import MultiAgentEnv
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env

import numpy as np
import torch
import sys
import pickle
import time
import copy
import pandas as pd
import os
import matplotlib.pyplot as plt
sys.path.insert(0,os.path.join(os.getcwd(), 'src')) ## Make ciceroscm importable

from ciceroscm import CICEROSCM
from ciceroscm.parallel.cscmparwrapper import run_ciceroscm_parallel
import ciceroscm.input_handler as input_handler
from train import load_processed_data, format_data
from model import LSTMSurrogate
from data_generation import load_core_data

## 0. Prepare CICERO-SCM for env step

In [29]:
cfg = [
    {
        "pamset_udm": {
            "rlamdo": 15.08357,
            "akapa": 0.6568376339229769,
            "cpi": 0.2077266,
            "W": 2.205919,
            "beto": 6.89822,
            "lambda": 0.6062529,
            "mixed": 107.2422,
        },
        "pamset_emiconc": {
            "qbmb": 0.0,
            "qo3": 0.5,
            "qdirso2": -0.3562,
            "qindso2": -0.96609,
            "qbc": 0.1566,
            "qoc": -0.0806,
        },
        "Index": "13555_old_NR_improved",
    }
]

conc_data_first = 1750
conc_data_last = 2100

em_data_start = 1900
em_data_policy = 2015
em_data_end = 2015

gaspam_data, conc_data, em_data, nat_ch4_data, nat_n2o_data = load_core_data()

historical_emissions = em_data.loc[:2015]
baseline_emissions = em_data.loc[2015+1:]

test_data_dir = "/home/obola/repositories/cicero-scm-surrogate/ciceroscm/tests/test-data"

baseline_scenario = {
            "gaspam_data": gaspam_data,
            "nyend": em_data_end,
            "nystart": em_data_start,
            "emstart": em_data_policy,
            "concentrations_data": conc_data,
            "nat_ch4_data": nat_ch4_data,
            "nat_n2o_data": nat_n2o_data,
            "emissions_data": historical_emissions,      
            "udir": test_data_dir,
            "idtm": 24,
            "scenname": "baseline_scenario",
        }

cscm_dir=CICEROSCM(baseline_scenario)
t0 = time.time()
cscm_dir._run({"results_as_dict":True}, pamset_udm=cfg[0]["pamset_udm"], pamset_emiconc=cfg[0]["pamset_emiconc"])
t1 = time.time()

print(f"Time: {t1-t0}")
print(f"Temperature: {cscm_dir.results['dT_glob_air'][-1]}")

Time: 0.3404531478881836
Temperature: 0.7520427266712681


In [46]:
class CICEROSCMEngine:
    """
    Reference engine that consumes full history 1900..t (G=40),
    calls CICEROSCM each step, and returns next-year temperature.

    Usage:
        eng = CICEROSCMEngine(
            historical_emissions=historical_emissions_np,
            gaspam_data=gaspam_data,
            conc_data=conc_data,
            nat_ch4_data=nat_ch4_data,
            nat_n2o_data=nat_n2o_data,
            pamset_udm=cfg[0]["pamset_udm"],
            pamset_emiconc=cfg[0]["pamset_emiconc"],
            em_data_start=1900,
            em_data_policy=2015,
            udir=test_data_dir,  # your output dir
            idtm=24,
            scenname="rl_scenario",
        )
        T, info = eng.step(E_t)   # E_t shape (G,)
    """
    def __init__(
        self,
        historical_emissions,
        gaspam_data,
        conc_data,
        nat_ch4_data,
        nat_n2o_data,
        pamset_udm,
        pamset_emiconc,
        em_data_start=1900,
        em_data_policy=2015,
        udir=".",
        idtm=24,
        scenname="rl_scenario",
    ):
        # ----- store static inputs -----
        self.gaspam_data = gaspam_data
        self.conc_data = conc_data
        self.nat_ch4_data = nat_ch4_data
        self.nat_n2o_data = nat_n2o_data
        self.pamset_udm = pamset_udm
        self.pamset_emiconc = pamset_emiconc
        self.udir = udir
        self.idtm = int(idtm)
        self.scenname = str(scenname)

        self.em_df = historical_emissions
        self.current_year = em_data_policy
        self.T = 0.0  

        # ----- immutable scenario template -----
        self._scenario_template = {
            "gaspam_data": self.gaspam_data,
            "nyend": self.current_year,
            "nystart": int(em_data_start),
            "emstart": int(em_data_policy),
            "concentrations_data": self.conc_data,
            "nat_ch4_data": self.nat_ch4_data,
            "nat_n2o_data": self.nat_n2o_data,
            "emissions_data": self.em_df,  # will be replaced each step
            "udir": self.udir,
            "idtm": self.idtm,
            "scenname": self.scenname,
        }

    def _build_scenario(self):
        sc = copy.copy(self._scenario_template)
        sc["emissions_data"] = self.em_df
        sc["nyend"] = int(self.em_df.index[-1])
        return sc

    def step(self, E_t):
        """
        Append emissions E_t (shape (G,)) for year current_year+1,
        run CICEROSCM, and return (T_next, info).
        """
        next_year = int(self.current_year + 1)
        e = np.asarray(E_t, dtype=np.float32)

        # Append new year to emissions DF
        self.em_df = copy.copy(self.em_df)
        self.em_df.loc[next_year] = e
        self.current_year = next_year

        # Build scenario and run SCM (time just the engine)
        scenario = self._build_scenario()
        cscm = CICEROSCM(scenario)
        cscm._run({"results_as_dict": True},
                            pamset_udm=self.pamset_udm,
                            pamset_emiconc=self.pamset_emiconc)

        # Extract temperature for next_year
        T_next = cscm.results["dT_glob_air"][-1]

        self.T = float(T_next)
        return self.T

In [47]:
# Historical emissions matrix H_hist: shape (1900..2015, G)
# Keep column order (this must match your per-step E_t order!)
scm_engine = CICEROSCMEngine(
    historical_emissions=historical_emissions,
    gaspam_data=gaspam_data,
    conc_data=conc_data,
    nat_ch4_data=nat_ch4_data,
    nat_n2o_data=nat_n2o_data,
    pamset_udm=cfg[0]["pamset_udm"],
    pamset_emiconc=cfg[0]["pamset_emiconc"],
    em_data_start=em_data_start,
    em_data_policy=em_data_policy,
    udir=test_data_dir,
    idtm=24,
    scenname="scm_unit_test",
)

for i in range(5):
    print(f"Year: {i}")
    next_emission = copy.copy(baseline_emissions.iloc[i].values)
    t0 = time.time()
    T = scm_engine.step(next_emission)
    t1 = time.time()
    print(f"Time: {t1-t0}")
    print(f"Temperature: {T}")

Year: 0
Time: 0.39900779724121094
Temperature: 0.7945743213966652
Year: 1
Time: 0.406796932220459
Temperature: 0.8315334241383018
Year: 2
Time: 0.4093959331512451
Temperature: 0.8684077734213573
Year: 3
Time: 0.41803574562072754
Temperature: 0.90415658042691
Year: 4
Time: 0.446674108505249
Temperature: 0.9394239810981585


## 1. Prepare CICERO-NET as env step

In [48]:
# Load model
run_dir="/home/obola/repositories/cicero-scm-surrogate/data/20250805_152136"
device="cuda:0"
weights_name="model_lstm.pth"

model = LSTMSurrogate(n_gas=40, hidden=128, num_layers=1).to(device)
wpath = os.path.join(run_dir, weights_name)
state = torch.load(wpath, map_location=device, weights_only=False)
model.load_state_dict(state)
model.eval()
for p in model.parameters():
    p.requires_grad_(False)

cicero_net = model

In [49]:
# Get standardizer
run_path = "/home/obola/repositories/cicero-scm-surrogate/data/20250805_152136"
data_path = os.path.join(run_path, "processed")

X_train = np.load(os.path.join(data_path, "X_train.npy"))

gas_mu  = X_train.reshape(-1, 40).mean(axis=0)     # per-gas mean
gas_std = X_train.reshape(-1, 40).std(axis=0) + 1e-6

In [50]:
class CICERONetEngine:
    """
    Inference engine for a CICERO-NET model trained on inputs shaped (51, G):
      [50 past years; current year's emissions], target = T_{t+1}.

    Args
    ----
    historical_emissions : np.ndarray, shape (T_hist, G)
        Full history up to and including 2015. We keep the last 50 rows.
    model : torch.nn.Module
        Trained surrogate (expects (B, 51, G) and returns (B, 1) or (B,)).
    device : str
        "cuda:0" or "cpu".
    window : int
        Number of historical years (should be 50 to match training).
    mu, std : np.ndarray or torch.Tensor, shape (G,), optional
        Per-gas normalization used at training for the 51×G inputs.
        If None, no normalization is applied.
    autocast : bool
        Enable mixed precision (only meaningful on CUDA when use_half=True).
    use_half : bool
        Use float16 inside autocast; otherwise run in float32.
    """
    def __init__(self, historical_emissions, model, device="cuda:0", window=50,
                 mu=None, std=None, autocast=True, use_half=True):
        import torch
        self.W = int(window)
        self.device = torch.device(device)
        self.model = model.eval().to(self.device)
        self.use_half = bool(use_half) and (self.device.type == "cuda")
        # Only enable autocast if we’re actually using half precision.
        self.autocast = bool(autocast) and self.use_half

        # Fast kernels (safe no-ops on CPU)
        torch.backends.cudnn.benchmark = True
        try:
            torch.set_float32_matmul_precision("high")
        except Exception:
            pass

        # ---- keep exactly the last 50 rows (t-50..t-1) as the rolling buffer ----
        hist_tail = np.asarray(historical_emissions[-self.W:], dtype=np.float32)   # (50, G)
        self.G = hist_tail.shape[1]
        self.buf = torch.from_numpy(hist_tail).to(self.device, non_blocking=True)  # (50, G)

        # Model input is (1, 51, G)
        dtype = torch.float16 if self.use_half else torch.float32
        self.x = torch.empty((1, self.W + 1, self.G), device=self.device, dtype=dtype)

        # Optional per-gas normalization stats (on device, dtype-matched)
        self.mu  = None if mu  is None else torch.as_tensor(mu,  device=self.device, dtype=self.x.dtype)
        self.std = None if std is None else torch.as_tensor(std, device=self.device, dtype=self.x.dtype)

        self.T = 0.0  # last predicted temperature (scalar float)

    @torch.inference_mode()
    def step(self, E_t):
        """
        Feed [buf(50,G); E_t(1,G)] → model → T_{t+1}; then roll buf to include E_t.
        E_t : array-like, shape (G,)
        Returns:
            float temperature prediction for next year.
        """
        e = torch.as_tensor(E_t, device=self.device, dtype=self.buf.dtype)  # (G,)

        # Build (51, G) input: first the 50 past rows …
        self.x[0, :self.W].copy_(self.buf, non_blocking=True)
        # … then append current action-year emissions
        self.x[0, self.W].copy_(e, non_blocking=True)

        # Normalize inputs if stats provided: (x - mu)/std (broadcast over time)
        if (self.mu is not None) and (self.std is not None):
            self.x[0].sub_(self.mu).div_(self.std)

        # Forward pass
        if self.autocast:
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                out = self.model(self.x)
        else:
            out = self.model(self.x)

        T_next = float(out.squeeze().item())
        self.T = T_next

        # Update the rolling 50-year buffer for the next call: drop oldest, append E_t
        self.buf = torch.roll(self.buf, shifts=-1, dims=0)
        self.buf[-1].copy_(e, non_blocking=True)

        return self.T

In [51]:
net_engine = CICERONetEngine(historical_emissions = historical_emissions,
                             model = cicero_net,
                             device="cuda:0", 
                             window=50,
                             mu=gas_mu, 
                             std=gas_std, 
                             autocast=True, 
                             use_half=False)

for i in range(5):
    print(f"Year: {i}")
    next_emission = copy.copy(baseline_emissions.iloc[i].values)
    t0 = time.time()
    T = net_engine.step(next_emission)
    t1 = time.time()
    print(f"Time: {t1-t0}")
    print(f"Temperature: {T}")

Year: 0
Time: 0.0014331340789794922
Temperature: 0.7944691777229309
Year: 1
Time: 0.0009052753448486328
Temperature: 0.8315608501434326
Year: 2
Time: 0.0008723735809326172
Temperature: 0.8684326410293579
Year: 3
Time: 0.0008866786956787109
Temperature: 0.9042533040046692
Year: 4
Time: 0.0008101463317871094
Temperature: 0.9391978979110718


## 2. MARL environment

## 3. OLD MARL

In [52]:
class ClimateMARL(MultiAgentEnv):
    """
    N agents (countries). Each year they pick a deviation a_i from their baseline share.
    Total emissions E_t (40 gases) feed a climate engine (CICERO-NET or CICERO-SCM).
    Observation (same for all agents): [T_t, year_norm, B_t(40)].
    Reward_i = xxx
    """

    def __init__(self, env_config, emission_data, economics_config, actions_config):
        super().__init__()
        # --- config dicts ---
        rng = np.random.default_rng(env_config['random_seed'])
      
        # --- core sizes/time ---
        self.N = int(env_config["N"])
        self.G = int(env_config["G"])
        self.year0_hist = int(env_config["year0_hist"])
        self.hist_end   = int(env_config["hist_end"])
        self.future_end = int(env_config["future_end"])
        self.horizon    = int(env_config["horizon"])
        self.engine_kind = str(env_config['engine']).lower()
        
        # --- provided data ---
        self.historical_emissions  = np.asarray(emission_data['historical_emissions'], dtype=np.float32)
        self.baseline_emissions    = np.asarray(emission_data["baseline_emissions"], dtype=np.float32)
        self.emission_shares       = np.asarray(emission_data["emission_shares"], dtype=np.float32)

        # --- heterogeneous economics/impacts ---
        self.climate_disaster_cost         = np.asarray(economics_config['climate_disaster_cost'], dtype=np.float32)         # (N,)
        self.climate_investment_cost       = np.asarray(economics_config['climate_investment_cost'], dtype=np.float32)       # (N,)
        self.economic_growth_sensitivity   = np.asarray(economics_config['economic_growth_sensitivity'], dtype=np.float32)   # (N,)

        # --- actions ---
        self.actions = np.asarray(actions_config['m_levels'], dtype=np.float32)  # e.g. [-0.05, ..., 0.05]
        self._act_space = spaces.Discrete(len(self.actions))  # <-- use actions

        # --- agents  ---
        self.agents = [f"country_{i}" for i in range(self.N)]

        # --- spaces ---
        obs_low  = np.array([0, 0, ] + [0.0]*self.G, dtype=np.float32)
        obs_high = np.array([2.0, 35,] + [10000]*self.G, dtype=np.float32)
        self._obs_space = spaces.Box(low=obs_low, high=obs_high, dtype=np.float32)
        self.observation_spaces = {a: self._obs_space for a in self.agents}
        self.action_spaces      = {a: self._act_space for a in self.agents}

        # --- model parameters ---
        self.net_params = env_config.get("net_params", {})
        self.scm_params = env_config.get("scm_params", {})

        # internal state
        self.reset()

    
    def observation_space(self, agent_id=None):
        return self.observation_spaces[self.agents[0] if agent_id is None else agent_id]

    def action_space(self, agent_id=None):
        return self.action_spaces[self.agents[0] if agent_id is None else agent_id]

    def _make_engine(self):
        if self.engine_kind == "net":
            p = self.net_params
            # rebuild model from class path
            module_name, cls_name = p["model_class_path"].rsplit(".", 1)
            Mod = __import__(module_name, fromlist=[cls_name])
            ModelCls = getattr(Mod, cls_name)
            m = ModelCls(**p["model_kwargs"])
            m.load_state_dict(p["state_dict"])
            # construct the high-throughput engine
            return CICERONetEngine(
                historical_emissions=self.historical_emissions,
                model=m,
                device=p.get("device", "cpu"),
                window=int(p.get("window", 50)),
                mu=np.asarray(p["mu"], np.float32) if p.get("mu") is not None else None,
                std=np.asarray(p["std"], np.float32) if p.get("std") is not None else None,
                autocast=bool(p.get("autocast", False)),
                use_half=bool(p.get("use_half", False)),
            )
        elif self.engine_kind == "scm":
            p = self.net_params
            
            return CICEROSCMEngine(
                historical_emissions=self.historical_emissions,
                gaspam_data=p["gaspam_data"],
                conc_data=p["conc_data"],
                nat_ch4_data=p["nat_ch4_data"],
                nat_n2o_data=p["nat_n2o_data"],
                pamset_udm=p["pamset_udm"],
                pamset_emiconc=p["pamset_emiconc"],
                em_data_start=p["em_data_start"],
                em_data_policy=p["em_data_policy"],
                udir=p["udir"],
                idtm=p["idtm"],
                scenname=p["scenname"],
            )
        else:
            raise ValueError(self.engine_kind)

    def _obs(self):
        temp = float(self.engine.T)
        year_idx = float(self.year_idx - (self.hist_end + 1))  # 0..35
        emissions = self.historical_emissions[-1]              # (G,)
        return np.asarray([temp, year_idx, *emissions], dtype=np.float32)

    
    def reset(self, *, seed=None, options=None):
        if seed is not None:
            _ = np.random.default_rng(seed)
        self.t = 0
        self.year_idx = self.hist_end + 1  # 2016
        self.engine = self._make_engine()
        obs = self._obs()
        return {a: obs for a in self.agents}, {a: {} for a in self.agents}

    def step(self, action_dict):
        
        a_vec = np.array([self.actions[action_dict[ag]] for ag in self.agents], dtype=np.float32)  # (N,)
        
        B_t = self.baseline_emissions[self.year_idx - (self.hist_end + 1)]  # (40,)
        
        b_alloc = self.emission_shares * B_t[None, :]  # (N,40)
        
        e_agents = (1.0 + a_vec[:, None]) * b_alloc
        e_agents = np.clip(e_agents, 0.0, None)
        
        E_t = e_agents.sum(axis=0)  # (40,)
        
        self.historical_emissions = np.vstack([self.historical_emissions, E_t[None, :]])
        T_next = self.engine.step(E_t)

        # 6) rewards
        emiss_agent_sum = e_agents.sum(axis=1)   # (N,)
        base_agent_sum  = b_alloc.sum(axis=1)    # (N,)
        delta_emissions = a_vec * base_agent_sum   # (N,)

        # (A) Climate disasters cost: penalize high temperature (use T_t, i.e., pre-update)
        disaster_cost = self.climate_disaster_cost * (T_next ** 2)   # (N,)

        # (B) Climate investment cost: only when mitigating (a < 0).
        # Scale by the size of the baseline so deeper cuts cost more.
        mitigation_amount = np.clip(-a_vec, 0.0, None)                      # (-a) if a<0 else 0
        investment_cost  = self.climate_investment_cost * mitigation_amount * base_agent_sum  # (N,)

        # (C) Economic growth sensitivity: more emissions boosts growth (a>0), cuts reduce it (a<0).
        # Linear in delta emissions; you can make it nonlinear later if you want.
        growth_term = self.economic_growth_sensitivity * delta_emissions    # (N,)

        r = growth_term - investment_cost - disaster_cost                   # (N,)
        
        self.t += 1
        self.year_idx += 1
        done = (self.t >= self.horizon) or (self.year_idx > self.future_end)
        
        obs = self._obs()
        obs_d = {a: obs for a in self.agents}
        
        rew_d   = {a: float(x) for a, x in zip(self.agents, r)}
        term_d  = {a: done for a in self.agents}
        trunc_d = {a: False for a in self.agents}
        info_d  = {a: {"E_sum": float(E_t.sum())} for a in self.agents}
        term_d["__all__"]  = done
        trunc_d["__all__"] = False

        return obs_d, rew_d, term_d, trunc_d, info_d

In [53]:
# ----------------------------- Training with RLlib -----------------------------
# Example configuration
env_config = {"N": 4, 
              "engine": "scm", 
              "horizon": 35, 
              "G": 40, 
              "year0_hist": 1900, 
              "hist_end": 2015, 
              "future_end": 2050,
              "random_seed": 0,
              }

if env_config['engine'] == "net":
    # make a small, CPU state_dict to put in env_config
    state_dict_cpu = {k: v.cpu() for k, v in cicero_net.state_dict().items()}
    net_params = {
        "model_class_path": "model.LSTMSurrogate",
        "model_kwargs": {"n_gas": 40, "hidden": 128, "num_layers": 1},
        "state_dict": state_dict_cpu,     # <- picklable
        "device": "cuda:0",
        "window": 50,
        "mu": gas_mu.tolist() if gas_mu is not None else None,
        "std": gas_std.tolist() if gas_std is not None else None,
        "autocast": True,
        "use_half": False,
    }
    env_config["net_params"] = net_params

elif env_config['engine'] == "scm":
    net_params = {
        "gaspam_data": gaspam_data,
        "conc_data": conc_data,
        "nat_ch4_data": nat_ch4_data,
        "nat_n2o_data": nat_n2o_data,
        "pamset_udm": cfg[0]["pamset_udm"],
        "pamset_emiconc": cfg[0]["pamset_emiconc"],
        "em_data_start": em_data_start,
        "em_data_policy": em_data_policy,
        "udir": test_data_dir,
        "idtm": 24,
        "scenname": "scm_marl_test"
    }
    env_config["net_params"] = net_params


# Example 
emission_shares = np.ones((4, 40), dtype=np.float32) / 4.0  # replace with real data

emission_data = {
    "historical_emissions": historical_emissions,
    "baseline_emissions": baseline_emissions,
    "emission_shares": emission_shares}

# Economic parameters per country
economics_config = {
    "climate_disaster_cost":       [0.5, 0.7, 0.9, 0.85],  # cost of climate change per temperature increase (per country)
    "climate_investment_cost":     [0.5, 0.7, 0.9, 0.85],  # cost of climate investment
    "economic_growth_sensitivity": [0.5, 0.7, 0.9, 0.85],  # economic growth sensitivity to investment in prevention
    }

actions_config = {
    "m_levels": [-0.05, -0.03, -0.01, 0.0, 0.01, 0.03, 0.05]  # Deviation from baseline emission shares
    }

# Build a temp env to read spaces and N
tmp = ClimateMARL(env_config, emission_data, economics_config, actions_config)
obs_sp = tmp.observation_space("country_0")
act_sp = tmp.action_space("country_0")
N = env_config["N"]

# One policy per country (no sharing)
policies = {f"country_{i}": (None, obs_sp, act_sp, {}) for i in range(N)}
policy_mapping_fn = lambda agent_id, *_, **__: agent_id

def env_creator(cfg):
    # cfg is the single env_config RLlib passes.
    return ClimateMARL(
        cfg["env_config"],
        cfg["emission_data"],
        cfg["economics_config"],
        cfg["actions_config"],
    )

register_env("climate_marl", env_creator)

algo = (
    PPOConfig()
    .environment(
        env="climate_marl",
        env_config={
            "env_config": env_config,
            "emission_data": emission_data,
            "economics_config": economics_config,
            "actions_config": actions_config,
        },
    )
    .framework("torch")
    # ↓↓↓ switch to old stack so num_gpus works as expected
    .api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)
    .env_runners(num_env_runners=0, num_envs_per_env_runner=1)
    .training(model={"fcnet_hiddens": [64, 64]}, gamma=0.99, lr=2e-3, train_batch_size=512)
    .multi_agent(policies=policies, policy_mapping_fn=policy_mapping_fn)
    .resources(num_gpus=1)     
    .build()
)

for i in range(100):
    result = algo.train()
    mean_ret = (
        result.get("episode_reward_mean")
        or result.get("episode_return_mean")
        or result.get("env_runners", {}).get("episode_return_mean")
        or result.get("sampler_results", {}).get("episode_reward_mean")
    )
    print(f"iter {i}: mean_return={mean_ret if mean_ret is not None else 'n/a'}")

/home/obola/miniconda3/envs/cicero/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:520: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/obola/miniconda3/envs/cicero/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/obola/miniconda3/envs/cicero/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API i

AttributeError: 'numpy.ndarray' object has no attribute 'loc'